In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import torch_geometric
import torch_geometric.nn as gnn
import torch_geometric.data as gdata
import torch_geometric.datasets as gdatasets

In [4]:
cora_path = "pyg_datasets/cora"
cora = gdatasets.Planetoid(cora_path, "cora")
print (cora)
print (cora[0])

cora()
Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


In [5]:
enzymes_path = "pyg_datasets/enzymes"
enzymes = gdatasets.TUDataset(enzymes_path, "ENZYMES")
print (enzymes)
print (enzymes[0])
print (enzymes[1])

ENZYMES(600)
Data(edge_index=[2, 168], x=[37, 3], y=[1])
Data(edge_index=[2, 102], x=[23, 3], y=[1])


In [7]:
# enzymes_train = enzymes[:540]

In [7]:
# this is a example of using gnn.GCNConv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
import torch_geometric.data as gdata
import torch_geometric.datasets as gdatasets

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
cora_path = "pyg_datasets/cora"
cora = gdatasets.Planetoid(cora_path, "cora")
cora = cora[0]

class Net(nn.Module):
    def __init__(self, in_feature, hid_feature, out_feature):
        nn.Module.__init__(self)
        self.gcn_1 = gnn.GCNConv(in_feature, hid_feature)
        self.gcn_2 = gnn.GCNConv(hid_feature, out_feature)
    
    def forward(self, x, edge_index):
        x = self.gcn_1(x, edge_index)
        x = F.relu(x)
        x = self.gcn_2(x, edge_index)
        return F.log_softmax(x, dim=1)

net, cora = Net(1433, 16, 7).to(device), cora.to(device)
optimizer = optim.Adam(net.parameters(), lr=1e-2)

for i in range(100):
    optimizer.zero_grad()
    logit = net(cora.x, cora.edge_index)[cora.train_mask]
    loss = F.nll_loss(logit, cora.y[cora.train_mask])
    loss.backward()
    optimizer.step()
    pred = net(cora.x, cora.edge_index)[cora.test_mask]
    pred = pred.max(1)[1]
    acc = pred.eq(cora.y[cora.test_mask]).sum().item() / cora.test_mask.sum().item()
    print ("epoch: {}\t, loss: {:.4f}, test_acc: {:.4f}".format(i, loss, acc))


epoch: 0	, loss: 1.9417, test_acc: 0.4460
epoch: 1	, loss: 1.8264, test_acc: 0.4910
epoch: 2	, loss: 1.6927, test_acc: 0.5060
epoch: 3	, loss: 1.5415, test_acc: 0.4960
epoch: 4	, loss: 1.4023, test_acc: 0.5200
epoch: 5	, loss: 1.2664, test_acc: 0.5720
epoch: 6	, loss: 1.1343, test_acc: 0.6290
epoch: 7	, loss: 1.0119, test_acc: 0.6630
epoch: 8	, loss: 0.8993, test_acc: 0.6850
epoch: 9	, loss: 0.7953, test_acc: 0.7020
epoch: 10	, loss: 0.6980, test_acc: 0.7170
epoch: 11	, loss: 0.6072, test_acc: 0.7320
epoch: 12	, loss: 0.5234, test_acc: 0.7560
epoch: 13	, loss: 0.4471, test_acc: 0.7670
epoch: 14	, loss: 0.3788, test_acc: 0.7720
epoch: 15	, loss: 0.3184, test_acc: 0.7860
epoch: 16	, loss: 0.2660, test_acc: 0.7910
epoch: 17	, loss: 0.2213, test_acc: 0.7920
epoch: 18	, loss: 0.1838, test_acc: 0.7960
epoch: 19	, loss: 0.1527, test_acc: 0.7980
epoch: 20	, loss: 0.1270, test_acc: 0.7970
epoch: 21	, loss: 0.1058, test_acc: 0.7950
epoch: 22	, loss: 0.0883, test_acc: 0.7950
epoch: 23	, loss: 0.0

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
import torch_geometric.nn as gnn
import torch_geometric.data as gdata
import torch_geometric.datasets as gdatasets

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
enzymes_path= "pyg_datasets/enzymes"
enzymes = gdatasets.TUDataset(enzymes_path, "ENZYMES")
enzymes = enzymes.shuffle()  # shuffle the dtataset
enzymes_train = enzymes[:540]
enzymes_test = enzymes[540:]
enzymes_trainloader = gdata.DataLoader(enzymes_train, batch_size=60, shuffle=True)
enzymes_testloader = gdata.DataLoader(enzymes_test, batch_size=60, shuffle=False)

class Net(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.conv1 = gnn.GraphConv(3, 128)
        self.pool1 = gnn.TopKPooling(128, ratio=0.8)
        self.conv2 = gnn.GraphConv(128, 128)
        self.pool2 = gnn.TopKPooling(128, ratio=0.8)
        self.conv3 = gnn.GraphConv(128, 128)
        self.pool3 = gnn.TopKPooling(128, ratio=0.8)

        self.lin1 = nn.Linear(256, 128)
        self.lin2 = nn.Linear(128, 64)
        self.lin3 = nn.Linear(64, 7)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gnn.global_max_pool(x, batch), gnn.global_mean_pool(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gnn.global_max_pool(x, batch), gnn.global_mean_pool(x, batch)], dim=1)
        
        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x3 = torch.cat([gnn.global_max_pool(x, batch), gnn.global_mean_pool(x, batch)], dim=1)
        
        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.log_softmax(self.lin3(x), dim=-1)

        return x

net = Net().to(device)
optimizer = optim.Adam(net.parameters(), lr=5e-4)

for i in range(100):  # one epoch
    for one_train_batch in enzymes_trainloader:
        net.train()
        optimizer.zero_grad()
        one_train_batch = one_train_batch.to(device)
        logit = net(one_train_batch.x, one_train_batch.edge_index, one_train_batch.batch)
        loss = F.nll_loss(logit, one_train_batch.y)
        loss.backward()
        optimizer.step()
    for one_test_batch in enzymes_testloader:
        net.eval()
        one_test_batch = one_test_batch.to(device)
        pred = net(one_test_batch.x, one_test_batch.edge_index, one_test_batch.batch)
        pred = pred.max(1)[1]
        acc = pred.eq(one_test_batch.y).sum().item() / len(one_test_batch.y)
    print ("epoch: {}\t, loss: {:.4f}, test_acc: {:.4f}".format(i, loss, acc))

epoch: 0	, loss: 1.9335, test_acc: 0.1000
epoch: 1	, loss: 1.8521, test_acc: 0.1667
epoch: 2	, loss: 1.8398, test_acc: 0.0833
epoch: 3	, loss: 1.8276, test_acc: 0.1500
epoch: 4	, loss: 1.8382, test_acc: 0.1833
epoch: 5	, loss: 1.8182, test_acc: 0.2000
epoch: 6	, loss: 1.7966, test_acc: 0.1667
epoch: 7	, loss: 1.8075, test_acc: 0.2167
epoch: 8	, loss: 1.7327, test_acc: 0.2000
epoch: 9	, loss: 1.7697, test_acc: 0.2333
epoch: 10	, loss: 1.7340, test_acc: 0.3667
epoch: 11	, loss: 1.7433, test_acc: 0.2833
epoch: 12	, loss: 1.7725, test_acc: 0.2333
epoch: 13	, loss: 1.6981, test_acc: 0.2333
epoch: 14	, loss: 1.7049, test_acc: 0.3667
epoch: 15	, loss: 1.7954, test_acc: 0.1833
epoch: 16	, loss: 1.5727, test_acc: 0.2333
epoch: 17	, loss: 1.6031, test_acc: 0.3333
epoch: 18	, loss: 1.7465, test_acc: 0.3500
epoch: 19	, loss: 1.5707, test_acc: 0.3000
epoch: 20	, loss: 1.7060, test_acc: 0.3167
epoch: 21	, loss: 1.7750, test_acc: 0.2500
epoch: 22	, loss: 1.5671, test_acc: 0.3000
epoch: 23	, loss: 1.5